In [1]:
from math import sqrt
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np

import preprocessing
import lstm
import postprocessing 
import RF
import linear


In [2]:

# Fetch data 

# input parameters for getData
startDateX = '2021-01-01'
endDateX = '2021-04-30'
startDateY = '2021-01-17'
endDateY = '2021-05-16'
geo = 'US-CA'
state = 'California'

# X is the google trends data and Y is the covid case number labels
X, Y = preprocessing.getData(startDateX, endDateX, startDateY, endDateY, geo, state)

# define train/test split over timeframe
train_percentage = 0.9
split_idx = round(len(Y)*train_percentage)


Fetching 'covid' for period:2020-08-04 2021-04-30
Fetching 'coronavirus' for period:2020-08-04 2021-04-30
Fetching 'covid-19' for period:2020-08-04 2021-04-30
Fetching 'covid cases' for period:2020-08-04 2021-04-30
Fetching 'coronavirus cases' for period:2020-08-04 2021-04-30
Fetching 'covid symptoms' for period:2020-08-04 2021-04-30
Fetching 'coronavirus symptoms' for period:2020-08-04 2021-04-30
Fetching 'cough' for period:2020-08-04 2021-04-30
Fetching 'virus' for period:2020-08-04 2021-04-30
Fetching 'vaccine' for period:2020-08-04 2021-04-30
Fetching 'covid vaccine' for period:2020-08-04 2021-04-30


In [20]:

# # Run linear regression model
# linear_predict = linear.linear(X, Y, train_percentage)
# linear_rms = postprocessing.RMSE(Y[split_idx:], linear_predict)

# # Run LSTM model 
# lstm_predict = lstm.lstm(X, Y, train_percentage)
# lstm_rms = postprocessing.RMSE(Y[split_idx:], lstm_predict)
# print(lstm_rms)

# Random Forest
# Train-Test Split for RF
RF_TTS_method = 1
if RF_TTS_method == 0:
    # Use the scikit learn train test split (random sampling)
    TTS_kwargs = { 'test_size': 1-train_percentage,
                'random_state': 0 } # Seed for random number generator
    train_features, test_features, train_labels, test_labels = preprocessing.RF_TTS(X, Y, TTS_kwargs)
else:
    # Use our manual method of TTS at split_idx
    train_features = X[:split_idx]
    test_features = X[split_idx:]
    train_labels = Y[:split_idx]
    test_labels = Y[split_idx:]
# Run RF model
RF_kwargs = {   'bootstrap': True,
                'criterion': 'squared_error',
                'max_depth': None, 
                'min_samples_leaf': 1,
                'n_estimators': 1000,
                'max_samples': 0.5, 
                'min_samples_split': 2, 
                'random_state': 0 } # Seed for random number generator
RFmodel, RF_predict = RF.randomForest(train_features, test_features, train_labels, RF_kwargs)

# postprocessing.plotTrainTest(Y, linear_predict, train_percentage, 'Linear Regression')

# # Postprocessing for LSTM
# postprocessing.plotTrainTest(Y, lstm_predict, train_percentage, 'LSTM')


In [21]:
numTrees = len(RFmodel.estimators_)
featureArray = np.zeros(numTrees)
for i in range(numTrees):
    featureArray[i] = RFmodel.estimators_[i].n_features_in_

np.mean(featureArray)


11.0

In [19]:
RFmodel.estimators_[0].n_features_in_

11

In [ ]:

# Postprocessing for RF
# Note: can probably use the same postprocessing function as with LSTM for this

postprocessing.plotTrainTest(Y, RF_predict, train_percentage, "Random Forest")

# feature_list = list(X.columns)
# features = np.array(X)
# postprocessing.plotRF(features, feature_list, Y, RF_predict)
# postprocessing.getRFImportances(RFmodel, feature_list)
# postprocessing.showAccuracyInfo(RF_predict, test_labels)
# postprocessing.plotRFTrees(RFmodel, feature_list, train_features, train_labels) # not working!!
